# `h5py` (direct, without xarray)

In [ ]:
import h5py
import fsspec
import earthaccess

In [ ]:
fsspec_kwargs = {
    "cache_type": "blockcache", 
    "block_size": 8*1024*1024
}

h5py_kwargs = {
    # "page_buf_size": 16*1024*1024,
    "rdcc_nbytes": 4*1024*1024
}

## Cloud

In [ ]:
# ISSUE REQUEST -- if you haven't run earthaccess.login() beforehand you
# will get an obtuse error
s3 = earthaccess.get_s3fs_session(daac="NSIDC")

url = 's3://nsidc-cumulus-prod-protected/ATLAS/ATL06/006/2019/12/02/ATL06_20191202203649_10220511_006_01.h5'

In [ ]:
%%timeit -r 5

with s3.open(url, 'rb', **fsspec_kwargs) as fo:
    with h5py.File(fo, **h5py_kwargs) as f:
        data0 = f['gt2l']['land_ice_segments']['h_li'][:]
        data1 = f['gt2l']['land_ice_segments']['latitude'][:]
        data2 = f['gt2l']['land_ice_segments']['longitude'][:]
        data3 = f['gt2l']['land_ice_segments']['delta_time'][:]

In [ ]:
(data0.nbytes + data1.nbytes + data2.nbytes + data3.nbytes) / 1e6

## Local

In [ ]:
fs = earthaccess.get_fsspec_https_session()

https = 'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ATLAS/ATL06/006/2019/12/02/ATL06_20191202203649_10220511_006_01.h5'

In [ ]:
%%timeit -r 5

with fs.open(https, 'rb', **fsspec_kwargs) as fo:
    with h5py.File(fo, **h5py_kwargs) as f:
        data0 = f['gt2l']['land_ice_segments']['h_li'][:]
        data1 = f['gt2l']['land_ice_segments']['latitude'][:]
        data2 = f['gt2l']['land_ice_segments']['longitude'][:]
        data3 = f['gt2l']['land_ice_segments']['delta_time'][:]
    # print(fo.cache)

print((data0.nbytes + data1.nbytes + data2.nbytes + data3.nbytes) / 1e6, 'MB loaded')